In [90]:
from google.oauth2 import service_account
import pandas_gbq as gbq
import json
from pandas_gbq import to_gbq, read_gbq
import re
from datetime import datetime, timezone
from textblob import TextBlob
import numpy as np
import pandas as pd
import requests



In [91]:
credentials_path = 'token.json'

# Authenticate with your credentials
credentials = service_account.Credentials.from_service_account_file(
    credentials_path, scopes=['https://www.googleapis.com/auth/bigquery'])

# Set the credentials for pandas_gbq
gbq.context.credentials = credentials

In [92]:
import pandas as pd
movies_data = pd.read_csv('movies.csv')

In [93]:
movies_data['title'] = movies_data['title'].str.replace("'", "")
movies_data['title']

movie_titles = []
movie_years = []

# Regular expression to extract title and year
pattern = re.compile(r"(.*) \((\d{4})\)")

for entry in movies_data['title']:
    match = pattern.match(entry)
    if match:
        movie_titles.append(match.group(1))
        movie_years.append(int(match.group(2)))  # Convert year to integer

(movie_titles, movie_years)

(['Toy Story',
  'Jumanji',
  'Grumpier Old Men',
  'Waiting to Exhale',
  'Father of the Bride Part II',
  'Heat',
  'Sabrina',
  'Tom and Huck',
  'Sudden Death',
  'GoldenEye',
  'American President, The',
  'Dracula: Dead and Loving It',
  'Balto',
  'Nixon',
  'Cutthroat Island',
  'Casino',
  'Sense and Sensibility',
  'Four Rooms',
  'Ace Ventura: When Nature Calls',
  'Money Train',
  'Get Shorty',
  'Copycat',
  'Assassins',
  'Powder',
  'Leaving Las Vegas',
  'Othello',
  'Now and Then',
  'Persuasion',
  'City of Lost Children, The (Cité des enfants perdus, La)',
  'Shanghai Triad (Yao a yao yao dao waipo qiao)',
  'Dangerous Minds',
  'Twelve Monkeys (a.k.a. 12 Monkeys)',
  'Wings of Courage',
  'Babe',
  'Carrington',
  'Dead Man Walking',
  'Across the Sea of Time',
  'It Takes Two',
  'Clueless',
  'Cry, the Beloved Country',
  'Richard III',
  'Dead Presidents',
  'Restoration',
  'Mortal Kombat',
  'To Die For',
  'How to Make an American Quilt',
  'Seven (a.k.a. Se7e

In [94]:
#push data to gbq
with open('token.json', 'r') as token_file:
    token_data = json.load(token_file)
    project_id = token_data.get('project_id', 'default-project-id')

dataset_id = "bt4240"
table_id = 'movies_data'

to_gbq(movies_data, destination_table=f'{project_id}.{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')

GenericGBQException: Reason: 403 POST https://bigquery.googleapis.com/bigquery/v2/projects/steadfast-helix-415216/datasets?prettyPrint=false: Access Denied: Project steadfast-helix-415216: User does not have bigquery.datasets.create permission in project steadfast-helix-415216.

In [108]:
from typing import *
import json
import praw
import pandas as pd

reddit = praw.Reddit(
    client_id='Rlmvoi5snEil5RLoeCymJw',
    client_secret='oMmJOOfBMoCjtROy4WDOikhvCxF-zw',
    user_agent='foosh'
)

SUBREDDIT_LIST = ['movies']

def setup_praw_reddit():
    with open(f'reddit.json', 'r') as f:
        key_file = json.load(f)
    try:
        client_id = key_file['client_id']
        client_secret = key_file['client_secret']
        user_agent = key_file['user_agent']
    except Exception as e:
        print(f'Error in retrieving required fields from key file: {e}')
        raise e
    try:
        reddit = praw.Reddit(
            client_id=client_id,
            client_secret=client_secret,
            user_agent=user_agent
        )
        return reddit
    except Exception as e:
        print(f'Error in setting up praw for reddit: {e}')
        raise e
    

def get_reddit_post(movie: str, subreddit_list: List[str] = SUBREDDIT_LIST) -> List[Dict[str, Any]]:
    reddit = setup_praw_reddit()
    reddit_list = []
    for subreddit in subreddit_list:
        try:
            praw = reddit.subreddit(subreddit)
            posts = praw.search(movie, sort='all', time_filter='all')

            for submission in posts:
                if submission.url[-3:] == "jpg" or submission.url[-3:] == "png":
                    continue
                reddit_list.append({
                    'movie_title': movie,
                    'subreddit': subreddit,
                    'id': submission.id,
                    'title': submission.title,
                    'url': submission.url,
                    'upvotes': submission.score,
                    'num_comments': submission.num_comments,
                    'author': submission.author.name,
                    'created_time': submission.created_utc # timestamp
                })
        except Exception as e:
            print(e)
    return reddit_list

def remove_punctuations(text):
    punct =['%','/',':','\\','&amp;','&',';', '?']
    for punctuation in punct:
        text = text.replace(punctuation, '')
    return text

def process_df(df, type):
    if type == 'comment':
        df['body'] = df['body'].apply(lambda x: remove_punctuations(x))
        df2 = df[df.body != '']

        df3 = df2.reset_index()
        df4 = df3[['body']]
        df4.rename(columns = {'body':'text'}, inplace = True)

    elif type == "submission":
        df['selftext'] = df['selftext'].apply(lambda x: remove_punctuations(x))
        df2 = df[df.selftext != '']

        df3 = df2.reset_index()
        df4 = df3[['selftext']]
        df4.rename(columns = {'selftext':'text'}, inplace = True)
    return df

def get_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def get_polarity(text):
    return TextBlob(text).sentiment.polarity

# df = process_df(df)
def process_textblob(df):
    df['subjectivity'] = df['text'].apply(get_subjectivity)
    df['polarity'] = df['text'].apply(get_polarity)
    # Obtain polarity scores generated by TextBlob
    df['textblob_score'] = df['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    # Convert polarity score into sentiment categories
    # neutral_threshold = 0.05
    # df['textblob_sentiment'] = df['textblob_score'].apply(lambda c: 'Positive' if c >= neutral_threshold else ('Negative' if c <= -(neutral_threshold) else 'Neutral'))

    return df

def avg_textblob(df):
    return df['textblob_score'].mean()

def process_text(df):
    """
    Perform sentiment analysis on the text data and add sentiment scores.
    """
    # Apply sentiment analysis on the title column
    df['sentiment'] = df['title'].apply(lambda x: TextBlob(x).sentiment.polarity)
    return df

def main():
    sentiment_scores = []

    for movie_title in movie_titles:  # Assuming movie_titles is a list of movie titles you want to analyze
        reddit_posts = get_reddit_post(movie_title)
        if not reddit_posts:  # If no posts found, skip to the next movie
            continue
        df = pd.DataFrame(reddit_posts)
        df = process_text(df)
        
        # Calculate the average sentiment score for the movie
        avg_sentiment_score = df['sentiment'].mean()
        print(f"Movie: {movie_title}, Average Sentiment: {avg_sentiment_score}")
        sentiment_scores.append({
            'movie_title': movie_title,
            'average_sentiment': avg_sentiment_score
        })

    return sentiment_scores

In [102]:
reddit = setup_praw_reddit()
praw = reddit.subreddit("movies")
posts = praw.search("The Matrix", sort='all', time_filter='all')
print(posts)


In [109]:
main()

KeyboardInterrupt: 